**Forewords**

- This script is a Jupyter notebook, mixing Markdown narratives with Python code chunks
	- Python is the most popular programming language in the world and the backbone of Arches
- This script is hosted on GitHub, it is mirrored here on Google colab: each changes made on the GitHub script will appear here
	- the script can also be download as an independant copy
- Google colab platform offer free space and computing time for online collaborative development of Jupyter scripts
	- another option could be MyBinder

The purpose of this presentation is:
1. show the join use of Jupyter/Python and GitHub to query the DB for users having intermediate skills in IT (or Arches)
2. collect ideas on Enhanced record minimum standard (ERMS)

# Enhanced record minimum standard compliance of Heritage Places

Enhanced record minimum standard (ERMS) is the minimum standard of data enhancement for heritage places. The report of Heritage Places ERMS is done downstream, once the heritages places (HP) have been recorded in the database 

### Import libraries

In [26]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.express as px

### Constants

Load:
- the UUID of HP in its resource model (RM)
- the read-only user `eamenar` parameters (see: [creating-a-read-only-user](https://github.com/eamena-project/eamena-arches-dev/tree/main/dev/postgres#creating-a-read-only-user)) on the training EAMENA instance
- ...

In [27]:
# Heritage Place Resource Model UUID
uuid_hp = '34cfe992-c2c0-11ea-9026-02e7594ce0a0'
# connection parameters
dbname = "eamena"
user = "eamenar"
password = "eamenar"
host = "52.50.27.140"
port = "5432"
# verbose
verbose = False

### Connect the database

In [40]:
try:
    connection = pg.connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )
    cur = connection.cursor()
    if verbose:
        print("Connection established successfully!")
except pg.Error as e:
    print(f"Error: {e}")

Connection established successfully!


## Heritage place selection

Select an HP and get its UUID (default: [EAMENA-0500002](http://52.50.27.140/search?paging-filter=1&tiles=true&format=tilecsv&reportlink=false&precision=6&total=3574&term-filter=%5B%7B%22context%22%3A%22%22%2C%22context_label%22%3A%22Heritage%20Place%20-%20EAMENA%20ID%22%2C%22id%22%3A0%2C%22text%22%3A%22EAMENA-0500002%22%2C%22type%22%3A%22term%22%2C%22value%22%3A%22EAMENA-0500002%22%2C%22inverted%22%3Afalse%7D%5D&language=*))

In [29]:
selected_hp = 'EAMENA-0500002'
sqll = """
SELECT
      resourceinstanceid AS resourceid
      FROM tiles
      WHERE tiledata -> '%s' -> 'en' ->> 'value' LIKE '%s'
""" % (uuid_hp, selected_hp)
# print(sqll)
cur.execute(sqll)
hpid = cur.fetchone()[0]  
if verbose: 
      print("the UUID of '" + selected_hp + "' is '" + hpid + "'")

## Heritage places field with their UUIDs

Read the [erms-template-readonly.tsv](https://github.com/eamena-project/eamena-arches-dev/blob/main/dev/data_quality/erms-template-readonly.tsv) file (see: [README.md](https://github.com/eamena-project/eamena-arches-dev/tree/main/dev/data_quality#erms)).

In [31]:
tsv_file = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/dev/data_quality/erms-template-readonly.tsv"
df = pd.read_csv(tsv_file, delimiter='\t')
df = df[["level1", "level2", "level3", "uuid_sql", "Enhanced record minimum standard"]]
df_listed = df.dropna()
if verbose:
    print(df_listed.to_markdown())

|    | level1                    | level2                            | level3                                     | uuid_sql                             | Enhanced record minimum standard                                                                                                                             |
|---:|:--------------------------|:----------------------------------|:-------------------------------------------|:-------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  1 | ASSESSMENT SUMMARY        | ASSESSMENT ACTIVITY               | Investigator Role Type                     | d2e1ab96-cc05-11ea-a292-02e7594ce0a0 | Yes                                                                                                                                                          |
|  5 | ASSESSMENT SUMMARY        | ASSESSMENT ACTIVITY    

Select the level of aggregation (`level1`, `level2` or `level3`) on which the spider plot will be done

In [36]:
options=['level1', 'level2', 'level3']
radio_button = widgets.RadioButtons(
    options=options,
    description='Select an option:'
)
display(radio_button)


RadioButtons(description='Select an option:', options=('level1', 'level2', 'level3'), value='level1')

Plot the ERMS dataframe for this level

In [38]:
# mylevel = 'level3'
mylevel = radio_button.value
df_erms = df_listed.copy()
df_erms['Enhanced record minimum standard'] = df_erms['Enhanced record minimum standard'].str.contains(r'Yes', case=False, na=False, regex=True).astype(int)
df_erms = df_erms[[mylevel, "Enhanced record minimum standard"]]
df_erms.columns.values[0] = "field"
df_erms = df_erms.groupby(['field'])['Enhanced record minimum standard'].sum()
print(f'You selected: {mylevel}')
print(df_erms.to_markdown())

You selected: level2
| field                             |   Enhanced record minimum standard |
|:----------------------------------|-----------------------------------:|
| ARCHAEOLOGICAL ASSESSMENT         |                                  1 |
| ASSESSMENT ACTIVITY               |                                  2 |
| CONDITION ASSESSMENT              |                                  0 |
| DESIGNATION                       |                                  0 |
| DISTURBANCES                      |                                  1 |
| GEOGRAPHY                         |                                  1 |
| GEOMETRIES                        |                                  0 |
| HERITAGE RESOURCE CLASSIFICATION  |                                  1 |
| PERIODIZATION                     |                                  1 |
| RESOURCE DESCRIPTION              |                                  0 |
| RESOURCE NAME                     |                                  0 |
| RE

Gather data from the HP and ERMS (creates an empty dataframe, loop over UUIDs to collect data from the selected HP {{selected_hp}}, and fill the empty dataframe)

In [41]:
# empty dataframe
level_values = df_listed[mylevel].unique()
data = {'field': level_values,
        'recorded': np.repeat(0, len(level_values)).tolist()}
df_res = pd.DataFrame(data)
# loop and fill it
for i in df_listed.index:
    if verbose:
        print("read: " + df_listed[mylevel][i] + ' | ' + df_listed['uuid_sql'][i])
    df_field = df_listed[mylevel][i]
    df_field_sql = re.sub(" ", "_", df_field) # rm space
    df_uuid = df_listed['uuid_sql'][i]
    sqll = """
    SELECT value FROM values 
    WHERE valueid::text IN
    (
    SELECT tiledata ->> '%s' AS %s
    FROM tiles 
    WHERE resourceinstanceid::text LIKE '%s'
    AND tiledata -> '%s' IS NOT NULL
    )
    """ % (df_uuid, df_field_sql, hpid, df_uuid)
    if verbose:
        print(sqll)
    cur.execute(sqll)
    outvalue = cur.fetchall()
    if len(outvalue) > 0:
        row_num = df_res[df_res['field'] == df_field].index.tolist()
        df_res.at[row_num[0], 'recorded'] = df_res.loc[row_num[0]]['recorded'] + 1
        if verbose:
            print("recorded values: " + str(outvalue))
if(verbose):
    print(df_res.to_markdown())

read: ASSESSMENT ACTIVITY | d2e1ab96-cc05-11ea-a292-02e7594ce0a0

    SELECT value FROM values 
    WHERE valueid::text IN
    (
    SELECT tiledata ->> 'd2e1ab96-cc05-11ea-a292-02e7594ce0a0' AS ASSESSMENT_ACTIVITY
    FROM tiles 
    WHERE resourceinstanceid::text LIKE 'dbc95d2d-38fb-465e-a6cb-0545eaa7584f'
    AND tiledata -> 'd2e1ab96-cc05-11ea-a292-02e7594ce0a0' IS NOT NULL
    )
    
read: ASSESSMENT ACTIVITY | b9643302-0407-11eb-a11c-0a5a9a4f6ef7

    SELECT value FROM values 
    WHERE valueid::text IN
    (
    SELECT tiledata ->> 'b9643302-0407-11eb-a11c-0a5a9a4f6ef7' AS ASSESSMENT_ACTIVITY
    FROM tiles 
    WHERE resourceinstanceid::text LIKE 'dbc95d2d-38fb-465e-a6cb-0545eaa7584f'
    AND tiledata -> 'b9643302-0407-11eb-a11c-0a5a9a4f6ef7' IS NOT NULL
    )
    
read: RESOURCE NAME | 34cfe9dd-c2c0-11ea-9026-02e7594ce0a0

    SELECT value FROM values 
    WHERE valueid::text IN
    (
    SELECT tiledata ->> '34cfe9dd-c2c0-11ea-9026-02e7594ce0a0' AS RESOURCE_NAME
    FROM tile

### Spider diagram

Show spider diagram with number of fields recorded. If `level3` has been selected, the spider plot will also plot the ERMS. 

In [46]:
tit = selected_hp + " - " + mylevel 
if mylevel == 'level3':
    colors = {'recorded': 'blue', 'Enhanced record minimum standard': 'red'}
    merged_df = pd.merge(df_res, df_erms, on='field')
    melted_df = pd.melt(merged_df, id_vars=['field'], var_name='Value Set', value_name='Value')
    melted_df.sort_values('Value Set', inplace=True)
    if verbose:
        print(melted_df.to_markdown())
    fig = px.line_polar(melted_df, r='Value', theta='field', color = 'Value Set',
                        line_close = False, color_discrete_map = colors, title = tit)
    fig.show()
else:
    variable = df_res['field'].tolist()
    value = df_res['recorded'].tolist()
    df = pd.DataFrame(dict(
        value = value,
        variable = variable))
    fig = px.line_polar(df, r = 'value', theta = 'variable', 
                        line_close = True, title = tit)
    fig.show()

## Development

* add a loop to work with 1..n HP
* improve the spider plot output (inetrvals, grid layout, etc.)
* connect the main DB

## Questions

* What kind of structure should we select to provide a list of HP (dataframe, list, etc.)?
* Do we want to have this ERMS assessement upstream (on the BU)?
* Do the code chunk are useful is this document, if not they can be grouped in functions, and these functions called from the Jupyter notebook (ex: `!python myfunction.py`)?